In [1]:
import sys, struct
import numpy as np
filename = "out.png"
fh = open(filename, 'rb')

first_eight = [137,80,78,71,13,10,26,10]
valid_count = 0;
i = 0
chunksize = 1
while (i < 8):
    first_byte = fh.read(chunksize)
    val = struct.unpack('B',first_byte)[0]
    if first_eight[i] == val:
        valid_count += 1
    else:
        print( "ERROR: Invalid PNG signature")
        exit(-1)
    i += 1;

if valid_count == 8:
    print( "Valid PNG signature")

Valid PNG signature


In [2]:
def messageToBinary(message):
    if type(message) == str:
        return ''.join([ format(ord(i), "08b") for i in message ])
    elif type(message) == bytes or type(message) == np.ndarray:
        return [ format(i, "08b") for i in message ]
    elif type(message) == int or type(message) == np.uint8:
        return format(message, "08b")
    else:
        raise TypeError("Input type not supported")

In [3]:
def process_ihdr():
    # The ! is necessary because PNGs being Portable "NETWORK" Graphics use
    # network byte ordering (which is big-endian). 
    width = struct.unpack('!I', fh.read(4))[0]
    print( "width: ", width)
    height = struct.unpack('!I', fh.read(4))[0]
    print( "height: ", height)
    bit_depth = struct.unpack('B', fh.read(1))[0]
    colour_type = struct.unpack('B', fh.read(1))[0]
    compression_method = struct.unpack('B', fh.read(1))[0]
    filter_method = struct.unpack('B', fh.read(1))[0]
    interlace_method = struct.unpack('B', fh.read(1))[0]
    print( "bit_depth: ", bit_depth)
    print( "colour_type: ", colour_type)
    print( "compression_method: ", compression_method)
    print ("filter_method: ", filter_method)
    print( "interlace_method: ", interlace_method)
    fh.read(4)

In [4]:
def process_punk(chunk_length):
    width = fh.read(chunk_length)
    #width = messageToBinary(width)
    width = width.decode('utf-8')
    print("ok?: ", width)

In [5]:
idats_found = 0
idats_bytes = 0
while (True):
    chlen = fh.read(4)
    if (len(chlen) == 0):
        break

    chunk_length = struct.unpack('!I',chlen)[0]

    chunk_type = ""
    for i in range(4):
        s=struct.unpack('c',fh.read(1))[0]
        chunk_type += (s.decode('UTF-8'))
    # Instead of printing the same report for each IDAT chunk, just
    # print a summary at the end, to reduce flooding your console
    if (chunk_type != "IDAT"):
        print( "=" * 20)
        print( "chunk_length: " + str(chunk_length)  )  
        print( "chunk_type: ", chunk_type)
    else:
        idats_found += 1
        idats_bytes += chunk_length

    if (chunk_type == "IHDR"):
        process_ihdr()
    elif(chunk_type == "puNK"):
        process_punk(chunk_length)
    else:
        data = fh.read(chunk_length)
        crc = fh.read(4)

print( "=" * 20)
print( "Found a total of " + str(idats_found) + " IDAT chunks consuming " + str(idats_bytes) + " bytes.")

chunk_length: 13
chunk_type:  IHDR
width:  512
height:  512
bit_depth:  8
colour_type:  2
compression_method:  0
filter_method:  0
interlace_method:  0
chunk_length: 24
chunk_type:  puNK
ok?:  000100101011100001100100
chunk_length: 1592857307
chunk_type:      
Found a total of 1 IDAT chunks consuming 632844 bytes.
